# Importing Libraries

I'm like 90% sure that everything is here, and that I don't have any unused libraries. 

In [ ]:
import pandas as pd
import numpy as np
import spacy
!spacy download en_core_web_sm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import string
import re
!pip install pyLDAvis==2.1.2
import pyLDAvis.gensim
from tqdm import tqdm
import warnings; warnings.simplefilter('ignore')
import time

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.6MB 4.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=e23793f6a278b0ee51c2a0043d5e5de6d6fe771b42267f2764b1ccac12645eb6
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


# Connecting to Google Drive


## Run This

In [ ]:
def google_authenticate():
  # Authenticate first so the Google Drive library can detect your credentials.
  from google.colab import auth
  auth.authenticate_user()

  from googleapiclient.discovery import build
  drive_service = build('drive', 'v3')
  return drive_service

## Then run this cell

Follow the instructions. None of that is saved to this notebook or by me or anyone really other than google, it's google's own auth system. 

In [ ]:
drive_service = google_authenticate()

## Run this cell too

In [ ]:
def read_file(file_id):
  """ 
  Download file from Google Drive 
  Argument: file_id
  Returns: downloaded file
  """
  
  file_id = file_id

  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

  downloaded.seek(0)
  return downloaded
  #print 'Downloaded file contents are:', downloaded.read()

## Actually Loading the Dataset
Paste the fileID of the file you've uploaded in-place of the string that I have here. 



> Hint: it's the first long string of random looking letters in the link to the file if you right-click on the file in your drive and select 'get link'



In [ ]:
text_file = read_file('1tPUBJCYHqSY53IgvgOsm1kzzvQEh4K4Y')

# Examining our Dataset

Sometimes it's helpful to take a look at the dataset you're working with. If you've already done all that, you can skip ahead after you've loaded the dataframe.

## Loading a Dataframe

Panda panda panda. 

In [ ]:
df = pd.read_csv(text_file)

## Taking a Peak
Checking really quick if the data looks like what we want it to look like. 


In [ ]:
df.head(10)

,posts
0,It seems like a major issue with this approach...
1,Man there really are some piece of shit medica...
2,solid find
3,"Yeah, from what I've seen the average for an N..."
4,I think the other poster is conflating hyponat...
5,History -&gt; volume status -&gt; serum osm -&...
6,Yeah I had a great experience as an M3 rotatin...
7,For inpatient? Yikes. ADA won't like that
8,That was ours once too. Now it's No Touching! ...
9,Thanks for sharing. This resonated with me


## Checking the Size
Let's peak to see how big this dataset is

In [ ]:
df.shape

(1528955, 1)

## Paring down our Sample
Oof, that's big.

Let's try working with just a subset, otherwise all of this will take all day. 

In [ ]:
df2 = df.sample(n=10000)

# Topic Modeling

Finally! We get to do a topic model!

All credit goes to: https://thinkinfi.com/guide-to-build-best-lda-model-using-gensim-python/

I pretty much just use his code

## Cleaning

But first, we have a lot of data cleaning to do. So let's start with that. 

In [ ]:
df2['posts']=df2['posts'].apply(str)
df2['posts']=df2['posts'].str.lower()

nlp = spacy.load("en_core_web_sm")

# Load NLTK stopwords
stop_words = stopwords.words('english')
# Add some extra words in it if required
stop_words.extend(['from', 'subject', 'use','pron'])

# Convert into list
data = df2.posts.values.tolist()

### Cleaning data

# Remove Emails
data = [re.sub('S*@S*s?', '', sent) for sent in data]
# Remove single quotes
data = [re.sub("'", "", sent) for sent in data]
# Remove links
data = [re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', sent) for sent in data]
# Remove spaces and new lines???

### Lemmatization
data_lemma = []
for txt in tqdm(data, desc='Lemmatization', unit='Llemas'):
    lis = []
    doc = nlp(txt)
    for token in doc:
        lis.append(token.lemma_)
    data_lemma.append(' '.join(lis))

### Tokenization and gensim stopword removal

# You can look for all gensim stopwords by running -> 'gensim.parsing.preprocessing.STOPWORDS'

# Function to tokenize
# Also remove words whose length less than 3 (you can chang it)
def tokenization_with_gen_stop(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)

    return result

## Apply tokenization function
data_words = []
for txt in tqdm(data_lemma, desc='Tokenizing', unit='tokens'):
    data_words.append(tokenization_with_gen_stop(txt))

### NLTK Stopword removal (extra stopwords)

data_words_clean = []
for word in tqdm(data_words, desc='Stop Word Removal', unit='words'):
    wrd = []
    for w in word:
        if w not in stop_words:
            wrd.append(w)
    data_words_clean.append(wrd)

100%|██████████| 10000/10000 [00:00<00:00, 26277.43it/s]


## Creating Dictionary and Corpus

You create a dictionary of words to tokens so that you can make use of the data that the model spits out, otherwise it's all just numbers. 

In [ ]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words_clean)
# Print dictionary
print(dictionary.token2id)

## Create Term document frequency (corpus)
# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in data_words_clean]
print('Done')

{'action': 0, 'automatically': 1, 'available': 2, 'compose': 3, 'concern': 4, 'contact': 5, 'include': 6, 'information': 7, 'list': 8, 'medicalschool': 9, 'message': 10, 'moderator': 11, 'page': 12, 'perform': 13, 'post': 14, 'question': 15, 'read': 16, 'remove': 17, 'require': 18, 'rule': 19, 'sidebar': 20, 'submission': 21, 'subreddit': 22, 'title': 23, 'compare': 24, 'number': 25, 'program': 26, 'quality': 27, 'resident': 28, 'schedule': 29, 'anatomy': 30, 'anatomyi': 31, 'average': 32, 'block': 33, 'cadaver': 34, 'chance': 35, 'curriculum': 36, 'feel': 37, 'forget': 38, 'harder': 39, 'help': 40, 'instead': 41, 'learn': 42, 'lifetime': 43, 'like': 44, 'matter': 45, 'month': 46, 'screw': 47, 'shoot': 48, 'shot': 49, 'slightly': 50, 'thrill': 51, 'waste': 52, 'wish': 53, 'work': 54, 'center': 55, 'patient': 56, 'population': 57, 'uncommon': 58, 'urban': 59, 'cool': 60, 'deal': 61, 'denial': 62, 'develop': 63, 'donation': 64, 'drag': 65, 'important': 66, 'prioritizing': 67, 'realistic'

## It's Training Time
So it's possible that you could be running this on something other than google colab, in which case you may have more than 2 cores! You can run the first cell to check, but if you don't know or don't care, then this doesn't matter and you can just run the second cell and call it a day. 

Grab some coffee. Could take a while. 

In [ ]:
!cat /proc/cpuinfo

In [ ]:
start_time = time.time()
## Multicore LDA
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, 
                                                    num_topics = 10, 
                                                    id2word=dictionary,
                                                    random_state=100,
                                                    passes=10,
                                                    chunksize=1000,
                                                    eval_every=100)
# Saving trained model
lda_model.save('medschoolishard')
# Loading trained model
lda_model = gensim.models.ldamodel.LdaModel.load('medschoolishard')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))
print(lda_model.print_topics())

--- 67.22416758537292 seconds ---
[(0, '0.019*"patient" + 0.013*"come" + 0.012*"people" + 0.012*"know" + 0.011*"resident" + 0.011*"student" + 0.010*"think" + 0.009*"like" + 0.008*"tell" + 0.008*"time"'), (1, '0.041*"thank" + 0.016*"think" + 0.011*"look" + 0.010*"comment" + 0.010*"love" + 0.008*"good" + 0.008*"link" + 0.008*"deck" + 0.007*"word" + 0.006*"like"'), (2, '0.016*"surgery" + 0.013*"like" + 0.012*"rotation" + 0.011*"letter" + 0.009*"surgical" + 0.007*"medicine" + 0.007*"sure" + 0.006*"year" + 0.006*"program" + 0.006*"ortho"'), (3, '0.028*"time" + 0.028*"week" + 0.023*"hour" + 0.020*"work" + 0.017*"school" + 0.013*"like" + 0.011*"think" + 0.010*"people" + 0.009*"study" + 0.007*"scrub"'), (4, '0.018*"school" + 0.015*"year" + 0.013*"residency" + 0.013*"good" + 0.013*"work" + 0.012*"research" + 0.011*"student" + 0.011*"thing" + 0.011*"like" + 0.010*"medical"'), (5, '0.026*"like" + 0.021*"know" + 0.014*"think" + 0.012*"psych" + 0.010*"care" + 0.009*"medicine" + 0.008*"practice" + 0

## Evaluate the Model

How'd we do? Did we do things? Let's find out!

Lower **Perplexity** is better.
Higher **Coherence** is better.

Coherence is apparently what everyone cares more about for reasons that I only superficially understand so I won't even attempt to explain it any further.

In [ ]:
# Compute Perplexity Score
print('nPerplexity Score: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_words_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

nPerplexity Score:  -7.936926573791501
nCoherence Score:  0.6182570321794434


# Hyperparamter Tuning

So above, we ran the model with just the base hyperparamters. Here, we run different permutations of:
1. Topic Number (how many different things do they actually talk about?
2. alpha: topic density, how many topics are in each document
3. beta: word density, how many words do topics consist of

Note: For tqdm to work properly (the progress bars), the pbar total variable should be set to the number of iterations to be passed over which depents on the number of topic counts you're doing and the size of the steps for alpha and beta. Do some math. 

In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_clean, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 15
max_topics = 20
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm(total=300, desc='Models Tried', unit='models')
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

 83%|████████▎ | 300/360 [6:29:33<1:17:54, 77.91s/it] 


# Final Model Running

I can't show it here, but if you take a gander in the files, the above spits out a csv file where you can eyeball which combination fits your model best. 

So for better or worse, I'm going to assume that our hyperparamters scale and apply (according to my untrained eyeballs) them to a larger corpus of texts and that will be the model that we'll visualize. 

## Cleaning text

In [ ]:
df2 = df.sample(250000)
df2['posts']=df2['posts'].apply(str)
df2['posts']=df2['posts'].str.lower()

nlp = spacy.load("en_core_web_sm")

# Load NLTK stopwords
stop_words = stopwords.words('english')
# Add some extra words in it if required
stop_words.extend(['from', 'subject', 'use','pron'])

# Convert into list
data = df2.posts.values.tolist()

### Cleaning data

# Remove Emails
data = [re.sub('S*@S*s?', '', sent) for sent in data]
# Remove single quotes
data = [re.sub("'", "", sent) for sent in data]
# Remove links
data = [re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', sent) for sent in data]
# Remove spaces and new lines???

### Lemmatization
data_lemma = []
for txt in tqdm(data, desc='Lemmatization', unit='Llemas'):
    lis = []
    doc = nlp(txt)
    for token in doc:
        lis.append(token.lemma_)
    data_lemma.append(' '.join(lis))

### Tokenization and gensim stopword removal

# You can look for all gensim stopwords by running -> 'gensim.parsing.preprocessing.STOPWORDS'

# Function to tokenize
# Also remove words whose length less than 3 (you can chang it)
def tokenization_with_gen_stop(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)

    return result

## Apply tokenization function
data_words = []
for txt in tqdm(data_lemma, desc='Tokenizing', unit='tokens'):
    data_words.append(tokenization_with_gen_stop(txt))

### NLTK Stopword removal (extra stopwords)

data_words_clean = []
for word in tqdm(data_words, desc='Stop Word Removal', unit='words'):
    wrd = []
    for w in word:
        if w not in stop_words:
            wrd.append(w)
    data_words_clean.append(wrd)

Stop Word Removal: 100%|██████████| 250000/250000 [00:12<00:00, 20137.79words/s]


## Creating Dictionary and Corpus

In [ ]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words_clean)
# Print dictionary
print(dictionary.token2id)

## Create Term document frequency (corpus)
# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in data_words_clean]
print('Done')

{'clinical': 0, 'fine': 1, 'killer': 2, 'whoa': 3, 'worry': 4, 'year': 5, 'america': 6, 'average': 7, 'high': 8, 'condition': 9, 'doctor': 10, 'documentation': 11, 'good': 12, 'help': 13, 'honestly': 14, 'inadequate': 15, 'lifelong': 16, 'medical': 17, 'patient': 18, 'shitty': 19, 'think': 20, 'yeah': 21, 'count': 22, 'allow': 23, 'break': 24, 'imagine': 25, 'like': 26, 'match': 27, 'reapply': 28, 'rule': 29, 'interview': 30, 'positive': 31, 'process': 32, 'program': 33, 'receive': 34, 'reflect': 35, 'response': 36, 'send': 37, 'thank': 38, 'time': 39, 'week': 40, 'ymmv': 41, 'cheap': 42, 'google': 43, 'need': 44, 'place': 45, 'stay': 46, 'strong': 47, 'behaviour': 48, 'blatantly': 49, 'compare': 50, 'comparison': 51, 'different': 52, 'exactly': 53, 'fallout': 54, 'parameter': 55, 'potential': 56, 'predict': 57, 'scenario': 58, 'significant': 59, 'situation': 60, 'suicidal': 61, 'want': 62, 'point': 63, 'thread': 64, 'ccom': 65, 'change': 66, 'comlex': 67, 'community': 68, 'counterpart

## Training the Model

In [ ]:
start_time = time.time()
## Multicore LDA
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, 
                                                    num_topics = 19, 
                                                    id2word=dictionary,
                                                    random_state=100,
                                                    passes=10,
                                                    chunksize=1000,
                                                    alpha=.91,
                                                    eta=.91)
# Saving trained model
lda_model.save('medschoolishard')
# Loading trained model
lda_model = gensim.models.ldamodel.LdaModel.load('medschoolishard')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))
print(lda_model.print_topics())

--- 1754.9310710430145 seconds ---
[(0, '0.027*"month" + 0.020*"money" + 0.019*"live" + 0.018*"free" + 0.015*"worth" + 0.015*"plan" + 0.013*"loan" + 0.011*"option" + 0.011*"half" + 0.010*"area"'), (1, '0.107*"time" + 0.039*"tell" + 0.037*"resident" + 0.021*"hospital" + 0.018*"home" + 0.017*"attend" + 0.017*"leave" + 0.015*"spend" + 0.012*"attending" + 0.012*"break"'), (2, '0.076*"want" + 0.037*"medicine" + 0.033*"rotation" + 0.028*"research" + 0.025*"specialty" + 0.024*"surgery" + 0.024*"experience" + 0.021*"clinical" + 0.016*"field" + 0.014*"consider"'), (3, '0.016*"level" + 0.013*"sense" + 0.012*"difference" + 0.012*"cause" + 0.011*"increase" + 0.011*"explain" + 0.010*"normal" + 0.010*"result" + 0.009*"blood" + 0.009*"heart"'), (4, '0.046*"place" + 0.026*"thread" + 0.025*"mind" + 0.021*"usmle" + 0.014*"relate" + 0.012*"comlex" + 0.010*"focus" + 0.010*"brain" + 0.010*"cancel" + 0.009*"meme"'), (5, '0.051*"mean" + 0.051*"actually" + 0.045*"probably" + 0.033*"shit" + 0.028*"yeah" + 0.02

## Getting our Coherence Scores and Whatnot

Improvement! Who knows if it was the tuning or the larger corpus, but whatever. Let's take a look at the charts!

In [ ]:
# Compute Perplexity Score
print('nPerplexity Score: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_words_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

nPerplexity Score:  -8.10820938806969
nCoherence Score:  0.6517218545722689


# Visualize

For the

*a e s t h e t i c s*

In [ ]:
# To plot at Jupyter notebook
pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, 'medschoolishard.html')
plot

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.158742 -0.135847       1        1  7.530196
15    -0.174344  0.244500       2        1  7.263616
8     -0.100047 -0.123544       3        1  6.734433
2     -0.155898 -0.093274       4        1  6.442092
12    -0.097103  0.106508       5        1  6.297449
1      0.029297  0.031333       6        1  5.641425
6     -0.065641 -0.031646       7        1  5.624660
7     -0.006809 -0.041220       8        1  5.480102
14     0.036839  0.012894       9        1  5.427597
0     -0.022291 -0.020770      10        1  5.297826
11     0.084609 -0.021327      11        1  5.226450
9      0.057270  0.167094      12        1  4.905416
17     0.095250 -0.008454      13        1  4.904354
3      0.037794 -0.021937      14        1  4.688680
18     0.090748 -0.016432      15        1  4.120770
5      0.096969 -0.019550      16        1  4.118661
10     0.086523 -0.008898      17        1  3.662086
4      0.075170 -0.004722      18        1  3.343949
16     0.090407 -0.014708      19        1  3.290239, topic_info=           Term          Freq         Total Category  logprob  loglift
26         like  34771.000000  34771.000000  Default  30.0000  30.0000
81       school  31690.000000  31690.000000  Default  29.0000  29.0000
5          year  28164.000000  28164.000000  Default  28.0000  28.0000
20        think  25048.000000  25048.000000  Default  27.0000  27.0000
75         know  22728.000000  22728.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
2104       hide    352.355311    372.088173  Topic19  -5.8113   3.3597
1931     direct    525.442734    568.164034  Topic19  -5.4117   3.3360
3984  genuinely    414.952114    448.114448  Topic19  -5.6478   3.3373
6866       clue    350.869337    375.162572  Topic19  -5.8155   3.3473
2564    youtube    361.322655    491.287406  Topic19  -5.7862   3.1070

[765 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
159       1  0.000502     able
159       2  0.000502     able
159       3  0.986880     able
159       4  0.001757     able
159       5  0.000753     able
...     ...       ...      ...
2564     15  0.002035  youtube
2564     16  0.002035  youtube
2564     17  0.002035  youtube
2564     18  0.002035  youtube
2564     19  0.734804  youtube

[13832 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 16, 9, 3, 13, 2, 7, 8, 15, 1, 12, 10, 18, 4, 19, 6, 11, 5, 17])